In [9]:
import Pkg;
Pkg.status()

     Project cifar10conv v0.1.0
      Status `/Volumes/GoogleDrive/My Drive/UNOMIC/JuliaTesting/Julia/cifar10conv/Project.toml`
  [fbb218c0] BSON v0.3.5
  [587475ba] Flux v0.12.9
  [7073ff75] IJulia v1.23.2
  [86fae568] ImageView v0.10.15
  [4138dd39] JLD v0.13.1
  [eb30cadb] MLDatasets v0.5.15
  [91a5bcdd] Plots v1.26.0
  [de0858da] Printf


In [1]:
import Pkg;
Pkg.add("Zygote")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `/Volumes/GoogleDrive/My Drive/UNOMIC/JuliaTesting/Julia/cifar10conv/Project.toml`
  [e88e6eb3] + Zygote v0.6.36
  No Changes to `/Volumes/GoogleDrive/My Drive/UNOMIC/JuliaTesting/Julia/cifar10conv/Manifest.toml`


In [10]:
using Flux, MLDatasets
using Flux: onehotbatch, onecold, DataLoader, Optimiser
using BSON:@save,@load
# using JLD

In [11]:
function ConvMixer(in_channels, kernel_size, patch_size, dim, depth, N_classes)
    f = Chain(
            Conv((patch_size, patch_size), in_channels=>dim, gelu; stride=patch_size),
            BatchNorm(dim),
            [
                Chain(
                    SkipConnection(Chain(Conv((kernel_size,kernel_size), dim=>dim, gelu; pad=SamePad(), groups=dim), BatchNorm(dim)), +),
                    Chain(Conv((1,1), dim=>dim, gelu), BatchNorm(dim))
                ) 
                for i in 1:depth
            ]...,
            AdaptiveMeanPool((1,1)),
            flatten,
            Dense(dim,N_classes)
        )
    return f
end

ConvMixer (generic function with 1 method)

In [12]:
function get_data(batchsize; dataset = MLDatasets.CIFAR10, idxs = nothing)
    """
    idxs=nothing gives the full dataset, otherwise (for testing purposes) only the 1:idxs elements of the train set are given.
    """
    ENV["DATADEPS_ALWAYS_ACCEPT"] = "true" 

    # Loading Dataset
    if idxs==nothing
        xtrain, ytrain = dataset.traindata(Float32)
        xtest, ytest = dataset.testdata(Float32)
	else
        xtrain, ytrain = dataset.traindata(Float32, 1:idxs)
        xtest, ytest = dataset.testdata(Float32, 1:Int(idxs/10))
    end

    # Reshape Data to comply to Julia's (width, height, channels, batch_size) convention in case there are only 1 channel (eg MNIST)
    if ndims(xtrain)==3
        w = size(xtrain)[1]
        xtrain = reshape(xtrain, (w,w,1,:))
        xtest = reshape(xtest, (w,w,1,:))
    end
    
    ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

    train_loader = DataLoader((xtrain, ytrain), batchsize=batchsize, shuffle=true)
    test_loader = DataLoader((xtest, ytest), batchsize=batchsize)

    return train_loader, test_loader
end

get_data (generic function with 1 method)

In [13]:
function create_loss_function(dataloader, device)

    function loss(model)
        n = 0
        l = 0.0f0
        acc = 0.0f0

        for (x,y) in dataloader
            x,y = x |> device, y |> device
            z = model(x)        
            l += Flux.logitcrossentropy(z, y, agg=sum)
            acc += sum(onecold(z).==onecold(y))
            n += size(x)[end]
        end
        l / n, acc / n
    end

    return loss
   
end

create_loss_function (generic function with 1 method)

In [6]:
function train(n_epochs=2)

    #params: warning, the training can be long with these params
    print("Data Loading")
    train_loader, test_loader = get_data(128)
    print("Data Loading -- Done")
    η = 3e-4
    in_channel = 3
    patch_size = 2
    kernel_size = 7
    dim = 128
    dimPL = 2
    depth = 18
    use_cuda = false

    #logging the losses
    train_save = zeros(n_epochs, 2)
    test_save = zeros(n_epochs, 2)

    if use_cuda
        device = gpu
        @info "Training on GPU"
        print("GPU Select")
    else
        device = cpu
        @info "Training on CPU"
        print("CPU Select")
    end

    train_loss_fn = create_loss_function(train_loader, device)
    test_loss_fn = create_loss_function(test_loader, device)
    
    print("Model Compliling")
    model = ConvMixer(in_channel, kernel_size, patch_size, dim, depth, 10) |> device
    print("Model Compiled")
    
    ps = params(model)
    opt = Optimiser(
            WeightDecay(1f-3), 
            ClipNorm(1.0),
            ADAM(η)
            )
    
    print("- Training Start's - ")
    for epoch in 1:n_epochs
        for (x,y) in train_loader
            x,y = x|>device, y|>device
            gr = gradient(()->Flux.logitcrossentropy(model(x), y, agg=sum), ps)
            Flux.Optimise.update!(opt, ps, gr)
        end

        #logging
        train_loss, train_acc = train_loss_fn(model) |> cpu
        test_loss, test_acc = test_loss_fn(model) |> cpu
        train_save[epoch,:] = [train_loss, train_acc]
        test_save[epoch,:] = [test_loss, test_acc]
        
        
        
        @info "Epoch $epoch : Train loss = $train_loss || Validation accuracy = $test_acc."
#         if epoch%5==0
#             @info "Epoch $epoch : Train loss = $train_loss || Validation accuracy = $test_acc."
#         end

    end

    model = model |> cpu
#     @save "model.bson" model 
#     @save "losses.bson" train_save test_save
end



train (generic function with 2 methods)

In [22]:
if abspath(PROGRAM_FILE) == @__FILE__
    print("Hello")
end

In [23]:
abspath(PROGRAM_FILE)

"/Users/usmanasim/.julia/packages/IJulia/e8kqU/src/kernel.jl"

In [24]:
@__FILE__

"In[24]"

In [7]:
train()

Data LoadingData Loading -- DoneCPU SelectModel ComplilingModel Compiled- Training Start's - 

┌ Info: Training on CPU
└ @ Main In[6]:26


LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException: